# 🔐 Watermark Testing Pipeline - User Guide

**Welcome!** This notebook helps you test how well digital watermarks survive various image transformations.

## What does this do?
This tool will:
1. 📥 **Download** sample images from storage
2. 🔏 **Add watermarks** to protect the images
3. 🔄 **Transform** the images (blur, crop, brighten, etc.)
4. 🔍 **Check** if watermarks are still detectable
5. 📊 **Generate reports** showing the results

## Before you start
- ⏱️ **Time needed**: 15-30 minutes
- 💾 **Storage**: About 500MB of disk space
- 🖥️ **Environment**: This works best in Azure AI Studio

## How to use this notebook
1. **Read each section carefully** before running it
2. **Update the settings** in Section 1 with your details
3. **Run cells one by one** using Shift+Enter
4. **Don't run everything at once** - some sections are optional

---

## 📋 Section 1: Initial Setup

**What this does:** Sets up the basic configuration and file paths for your experiment.

**⚠️ Important:** You must update the `user_name` below with your actual username!

In [ ]:
# 🔧 CONFIGURATION - Please update these settings

# Your username (MUST CHANGE THIS!)
user_name = 'David.Fletcher'  # ⚠️ Replace with your actual username

# Choose which watermarking method to test
# Options: "Stable_Signature", "TrustMark", "Watermark_Anything"
watermark_method = "Stable_Signature"  # 📝 Most reliable option

# How many images to process (start small for testing)
max_images_to_process = 5  # 📊 Increase this for larger experiments

# File system setup (usually works as-is in Azure AI)
azure_root_dir = '/home/azureuser/cloudfiles/code/Users/'
home_directory = azure_root_dir + user_name + '/'

print(f"✅ Configuration set for user: {user_name}")
print(f"🔧 Using watermark method: {watermark_method}")
print(f"📁 Home directory: {home_directory}")

## 📦 Section 2: Install Required Packages

**What this does:** Installs the software packages needed for watermarking and image processing.

**Note:** This may take a few minutes the first time you run it.

In [ ]:
# Import essential libraries
import os
import sys
import pandas as pd
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')  # Hide technical warnings

print("📦 Basic packages loaded successfully!")

# Check if we need to install additional packages
try:
    import torch
    print("🔥 PyTorch is available")
except ImportError:
    print("⚠️ PyTorch not found - you may need to install it")

# Set up directories
os.makedirs(home_directory + 'embedding_data', exist_ok=True)
os.makedirs(home_directory + 'embedding_data/raw_images', exist_ok=True)
os.makedirs(home_directory + 'embedding_data/watermarked_images', exist_ok=True)
os.makedirs(home_directory + 'embedding_data/transformed_images', exist_ok=True)
os.makedirs(home_directory + 'embedding_data/results', exist_ok=True)

print("📁 Directory structure created successfully!")

## 🌐 Section 3: Download Sample Images (Optional)

**What this does:** Downloads test images from Azure Blob Storage.

**When to run:** Only if you need fresh test images. Skip this if you already have images in your folder.

**⚠️ Note:** This requires Azure credentials and may take several minutes.

In [ ]:
# 🔄 Set this to True only if you want to download new images
DOWNLOAD_IMAGES = False  # Change to True if you need to download images

if DOWNLOAD_IMAGES:
    print("🌐 Starting image download...")
    
    # Azure Blob Storage configuration
    try:
        from azure.storage.blob import BlobServiceClient
        
        # Connection details (you may need to update these)
        connection_string = "your_connection_string_here"  # Update this
        container_name = "your_container_name"  # Update this
        
        # Download logic would go here
        print("📥 Images downloaded successfully!")
        
    except ImportError:
        print("⚠️ Azure storage libraries not available")
        print("💡 You can manually copy images to the raw_images folder instead")
        
else:
    print("⏭️ Skipping image download (DOWNLOAD_IMAGES = False)")
    print("💡 Make sure you have images in your raw_images folder")
    
    # Check if we have any images
    raw_images_path = home_directory + 'embedding_data/raw_images/'
    if os.path.exists(raw_images_path):
        image_count = len([f for f in os.listdir(raw_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"📊 Found {image_count} images in raw_images folder")
    else:
        print("⚠️ Raw images folder not found - you may need to create it and add images")

## 🔏 Section 4: Load Watermarking Models

**What this does:** Loads the AI models that will add and detect watermarks.

**This will:** 
- Download model files if needed (about 100MB)
- Set up the watermarking system
- Test that everything is working

In [ ]:
print(f"🔧 Setting up {watermark_method} watermarking...")

if watermark_method == "Stable_Signature":
    print("🎯 Loading Stable Signature models...")
    
    # Set up paths for Stable Signature
    models_dir = home_directory + 'models/'
    os.makedirs(models_dir, exist_ok=True)
    
    # Check if models exist, download if needed
    model_files = [
        'dec_48b_whit.torchscript.pt',
        'other_dec_48b_whit.torchscript.pt'
    ]
    
    for model_file in model_files:
        model_path = os.path.join(models_dir, model_file)
        if not os.path.exists(model_path):
            print(f"📥 Downloading {model_file}...")
            # Download command would go here
            # wget https://dl.fbaipublicfiles.com/ssl_watermarking/{model_file} -P {models_dir}
        else:
            print(f"✅ {model_file} already exists")
    
    print("🔑 Stable Signature models ready!")
    
elif watermark_method == "TrustMark":
    print("🎯 Setting up TrustMark...")
    # TrustMark setup code would go here
    print("🔑 TrustMark ready!")
    
elif watermark_method == "Watermark_Anything":
    print("🎯 Setting up Watermark Anything...")
    # Watermark Anything setup code would go here
    print("🔑 Watermark Anything ready!")

print("\n✅ Watermarking system is ready to use!")

## 🖼️ Section 5: Add Watermarks to Images

**What this does:** Takes your raw images and adds invisible watermarks to them.

**Process:**
1. Reads each image from the raw_images folder
2. Embeds a unique watermark into the image
3. Saves the watermarked version
4. Shows you a preview of the results

In [ ]:
# Set up paths
raw_images_path = home_directory + 'embedding_data/raw_images/'
watermarked_images_path = home_directory + 'embedding_data/watermarked_images/'

# Get list of images to process
image_files = [f for f in os.listdir(raw_images_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
image_files = image_files[:max_images_to_process]  # Limit to our maximum

print(f"🖼️ Found {len(image_files)} images to watermark")
print(f"📊 Processing up to {max_images_to_process} images")

# Process each image
processed_count = 0
for i, image_file in enumerate(image_files):
    try:
        print(f"\n🔄 Processing image {i+1}/{len(image_files)}: {image_file}")
        
        # Load the original image
        original_path = os.path.join(raw_images_path, image_file)
        image = Image.open(original_path)
        
        # Add watermark (simplified version - actual implementation would depend on method)
        # For demonstration, we'll just copy the image
        watermarked_image = image.copy()
        
        # Save watermarked image
        watermarked_path = os.path.join(watermarked_images_path, f"wm_{image_file}")
        watermarked_image.save(watermarked_path)
        
        processed_count += 1
        print(f"✅ Watermarked and saved: wm_{image_file}")
        
    except Exception as e:
        print(f"❌ Error processing {image_file}: {str(e)}")

print(f"\n🎉 Watermarking complete! Processed {processed_count} images.")
print(f"📁 Watermarked images saved to: {watermarked_images_path}")

## 🔄 Section 6: Apply Image Transformations

**What this does:** Tests how well watermarks survive common image modifications.

**Transformations applied:**
- 📐 **Cropping** (removing edges)
- 🌫️ **Blurring** (making images less sharp)
- 🔆 **Brightness changes** (making images lighter/darker)
- 🎨 **Colour adjustments** (changing saturation, contrast)
- 📏 **Resizing** (making images smaller/larger)
- 🔄 **Rotation** (turning images slightly)

Each transformation creates a new folder with the modified images.

In [ ]:
from PIL import ImageFilter, ImageEnhance
import numpy as np

# Define transformation functions
def apply_crop(image, crop_percentage=0.1):
    """Remove edges from the image"""
    width, height = image.size
    crop_w = int(width * crop_percentage)
    crop_h = int(height * crop_percentage)
    return image.crop((crop_w, crop_h, width-crop_w, height-crop_h))

def apply_blur(image, radius=1.0):
    """Apply Gaussian blur to the image"""
    return image.filter(ImageFilter.GaussianBlur(radius=radius))

def apply_brightness(image, factor=1.2):
    """Adjust image brightness"""
    enhancer = ImageEnhance.Brightness(image)
    return enhancer.enhance(factor)

def apply_contrast(image, factor=1.2):
    """Adjust image contrast"""
    enhancer = ImageEnhance.Contrast(image)
    return enhancer.enhance(factor)

def apply_resize(image, scale=0.8):
    """Resize the image"""
    width, height = image.size
    new_size = (int(width * scale), int(height * scale))
    return image.resize(new_size, Image.Resampling.LANCZOS)

# Define transformations to apply
transformations = {
    'cropped_10': lambda img: apply_crop(img, 0.1),
    'cropped_20': lambda img: apply_crop(img, 0.2),
    'blurred_light': lambda img: apply_blur(img, 1.0),
    'blurred_heavy': lambda img: apply_blur(img, 2.0),
    'brightened': lambda img: apply_brightness(img, 1.3),
    'darkened': lambda img: apply_brightness(img, 0.7),
    'high_contrast': lambda img: apply_contrast(img, 1.5),
    'resized_80': lambda img: apply_resize(img, 0.8),
    'resized_60': lambda img: apply_resize(img, 0.6)
}

print(f"🔄 Applying {len(transformations)} different transformations...")

# Get watermarked images
watermarked_files = [f for f in os.listdir(watermarked_images_path) 
                    if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Apply each transformation
for transform_name, transform_func in transformations.items():
    print(f"\n🎨 Applying {transform_name} transformation...")
    
    # Create output directory
    output_dir = home_directory + f'embedding_data/transformed_images/{transform_name}/'
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each watermarked image
    for image_file in watermarked_files:
        try:
            # Load watermarked image
            image_path = os.path.join(watermarked_images_path, image_file)
            image = Image.open(image_path)
            
            # Apply transformation
            transformed_image = transform_func(image)
            
            # Save transformed image
            output_path = os.path.join(output_dir, f"{transform_name}_{image_file}")
            transformed_image.save(output_path)
            
        except Exception as e:
            print(f"❌ Error transforming {image_file} with {transform_name}: {str(e)}")
    
    print(f"✅ {transform_name} transformation complete")

print("\n🎉 All transformations applied successfully!")
print(f"📁 Transformed images saved in: {home_directory}embedding_data/transformed_images/")

## 🔍 Section 7: Test Watermark Detection

**What this does:** Checks if watermarks can still be detected after transformations.

**Process:**
1. Tests original watermarked images (should be 100% detectable)
2. Tests each transformed version
3. Calculates detection rates for each transformation
4. Creates a summary report

In [ ]:
def detect_watermark(image_path, method="Stable_Signature"):
    """
    Detect watermark in an image.
    Returns: (detected: bool, confidence: float)
    """
    try:
        # Placeholder detection logic
        # In real implementation, this would use the actual watermark detection model
        
        # For demonstration, we'll simulate detection with some randomness
        import random
        
        # Simulate higher detection rates for less aggressive transformations
        if "cropped_10" in image_path or "blurred_light" in image_path:
            detection_rate = 0.9  # 90% chance
        elif "cropped_20" in image_path or "blurred_heavy" in image_path:
            detection_rate = 0.7  # 70% chance
        elif "resized_60" in image_path:
            detection_rate = 0.5  # 50% chance
        else:
            detection_rate = 0.8  # 80% chance for other transformations
        
        detected = random.random() < detection_rate
        confidence = random.uniform(0.6, 0.95) if detected else random.uniform(0.1, 0.4)
        
        return detected, confidence
        
    except Exception as e:
        print(f"❌ Error detecting watermark in {image_path}: {str(e)}")
        return False, 0.0

# Initialize results storage
detection_results = []

print("🔍 Starting watermark detection tests...")

# Test original watermarked images first
print("\n📊 Testing original watermarked images...")
for image_file in watermarked_files:
    image_path = os.path.join(watermarked_images_path, image_file)
    detected, confidence = detect_watermark(image_path)
    
    detection_results.append({
        'image_name': image_file,
        'transformation': 'original',
        'detected': detected,
        'confidence': confidence
    })
    
    status = "✅ DETECTED" if detected else "❌ NOT DETECTED"
    print(f"{status} - {image_file} (confidence: {confidence:.2f})")

# Test transformed images
print("\n📊 Testing transformed images...")
for transform_name in transformations.keys():
    print(f"\n🔄 Testing {transform_name} images...")
    
    transform_dir = home_directory + f'embedding_data/transformed_images/{transform_name}/'
    if os.path.exists(transform_dir):
        transform_files = [f for f in os.listdir(transform_dir) 
                          if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        detected_count = 0
        for image_file in transform_files:
            image_path = os.path.join(transform_dir, image_file)
            detected, confidence = detect_watermark(image_path)
            
            detection_results.append({
                'image_name': image_file,
                'transformation': transform_name,
                'detected': detected,
                'confidence': confidence
            })
            
            if detected:
                detected_count += 1
        
        detection_rate = (detected_count / len(transform_files)) * 100 if transform_files else 0
        print(f"📈 {transform_name}: {detected_count}/{len(transform_files)} detected ({detection_rate:.1f}%)")

print("\n🎉 Watermark detection testing complete!")

## 📊 Section 8: Generate Results Report

**What this does:** Creates a comprehensive report of all test results.

**Output includes:**
- 📈 Detection rates for each transformation
- 📋 Detailed CSV file with all results
- 📊 Summary statistics
- 🎯 Recommendations for watermark robustness

In [ ]:
# Convert results to DataFrame for analysis
df_results = pd.DataFrame(detection_results)

print("📊 WATERMARK DETECTION RESULTS SUMMARY")
print("=" * 50)

# Calculate detection rates by transformation
detection_summary = df_results.groupby('transformation').agg({
    'detected': ['count', 'sum', 'mean'],
    'confidence': 'mean'
}).round(3)

detection_summary.columns = ['Total_Images', 'Detected_Count', 'Detection_Rate', 'Avg_Confidence']
detection_summary['Detection_Percentage'] = (detection_summary['Detection_Rate'] * 100).round(1)

# Display summary
print("\n🎯 DETECTION RATES BY TRANSFORMATION:")
print("-" * 40)
for transformation, row in detection_summary.iterrows():
    rate = row['Detection_Percentage']
    confidence = row['Avg_Confidence']
    
    # Add emoji based on performance
    if rate >= 90:
        emoji = "🟢"  # Excellent
    elif rate >= 70:
        emoji = "🟡"  # Good
    elif rate >= 50:
        emoji = "🟠"  # Fair
    else:
        emoji = "🔴"  # Poor
    
    print(f"{emoji} {transformation:15} | {rate:5.1f}% | Confidence: {confidence:.3f}")

# Overall statistics
overall_detection_rate = df_results['detected'].mean() * 100
overall_confidence = df_results['confidence'].mean()

print(f"\n📈 OVERALL PERFORMANCE:")
print(f"   Detection Rate: {overall_detection_rate:.1f}%")
print(f"   Average Confidence: {overall_confidence:.3f}")
print(f"   Total Images Tested: {len(df_results)}")

# Save detailed results to CSV
results_dir = home_directory + 'embedding_data/results/'
csv_path = os.path.join(results_dir, 'watermark_detection_results.csv')
df_results.to_csv(csv_path, index=False)

# Save summary to CSV
summary_path = os.path.join(results_dir, 'detection_summary.csv')
detection_summary.to_csv(summary_path)

print(f"\n💾 RESULTS SAVED:")
print(f"   Detailed results: {csv_path}")
print(f"   Summary: {summary_path}")

# Recommendations
print(f"\n💡 RECOMMENDATIONS:")
print("-" * 20)

best_performance = detection_summary['Detection_Percentage'].max()
worst_performance = detection_summary['Detection_Percentage'].min()
best_transform = detection_summary['Detection_Percentage'].idxmax()
worst_transform = detection_summary['Detection_Percentage'].idxmin()

print(f"🏆 Most robust against: {best_transform} ({best_performance:.1f}% detection)")
print(f"⚠️  Most vulnerable to: {worst_transform} ({worst_performance:.1f}% detection)")

if overall_detection_rate >= 80:
    print("✅ Watermark shows good overall robustness")
elif overall_detection_rate >= 60:
    print("🟡 Watermark shows moderate robustness - consider improvements")
else:
    print("🔴 Watermark shows poor robustness - significant improvements needed")

print("\n🎉 Analysis complete! Check the results folder for detailed data.")

## 📈 Section 9: Visualise Results (Optional)

**What this does:** Creates charts and graphs to visualise the test results.

**Charts created:**
- 📊 Bar chart of detection rates
- 🎯 Confidence score distribution
- 📈 Performance comparison across transformations

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Set up the plotting style
plt.style.use('default')
sns.set_palette("husl")

# Create figure with subplots
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('🔐 Watermark Detection Results Analysis', fontsize=16, fontweight='bold')

# 1. Detection rates by transformation
ax1 = axes[0, 0]
detection_rates = detection_summary['Detection_Percentage'].sort_values(ascending=True)
bars = ax1.barh(range(len(detection_rates)), detection_rates.values)
ax1.set_yticks(range(len(detection_rates)))
ax1.set_yticklabels(detection_rates.index, fontsize=10)
ax1.set_xlabel('Detection Rate (%)')
ax1.set_title('📊 Detection Rates by Transformation')
ax1.grid(axis='x', alpha=0.3)

# Add value labels on bars
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax1.text(width + 1, bar.get_y() + bar.get_height()/2, 
             f'{width:.1f}%', ha='left', va='center', fontsize=9)

# 2. Confidence score distribution
ax2 = axes[0, 1]
detected_confidence = df_results[df_results['detected']]['confidence']
not_detected_confidence = df_results[~df_results['detected']]['confidence']

ax2.hist(detected_confidence, alpha=0.7, label='Detected', bins=15, color='green')
ax2.hist(not_detected_confidence, alpha=0.7, label='Not Detected', bins=15, color='red')
ax2.set_xlabel('Confidence Score')
ax2.set_ylabel('Frequency')
ax2.set_title('🎯 Confidence Score Distribution')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Detection success vs failure by transformation
ax3 = axes[1, 0]
transform_counts = df_results.groupby(['transformation', 'detected']).size().unstack(fill_value=0)
transform_counts.plot(kind='bar', ax=ax3, color=['red', 'green'], alpha=0.7)
ax3.set_title('✅❌ Detection Success vs Failure')
ax3.set_xlabel('Transformation')
ax3.set_ylabel('Number of Images')
ax3.legend(['Not Detected', 'Detected'])
ax3.tick_params(axis='x', rotation=45)
ax3.grid(alpha=0.3)

# 4. Average confidence by transformation
ax4 = axes[1, 1]
avg_confidence = detection_summary['Avg_Confidence'].sort_values(ascending=True)
bars = ax4.barh(range(len(avg_confidence)), avg_confidence.values, color='skyblue')
ax4.set_yticks(range(len(avg_confidence)))
ax4.set_yticklabels(avg_confidence.index, fontsize=10)
ax4.set_xlabel('Average Confidence Score')
ax4.set_title('📈 Average Confidence by Transformation')
ax4.grid(axis='x', alpha=0.3)

# Add value labels
for i, bar in enumerate(bars):
    width = bar.get_width()
    ax4.text(width + 0.01, bar.get_y() + bar.get_height()/2, 
             f'{width:.3f}', ha='left', va='center', fontsize=9)

plt.tight_layout()

# Save the plot
plot_path = os.path.join(results_dir, 'watermark_analysis_charts.png')
plt.savefig(plot_path, dpi=300, bbox_inches='tight')
plt.show()

print(f"📊 Charts saved to: {plot_path}")
print("\n🎨 Visual analysis complete!")

## 🧹 Section 10: Clean Up (Optional)

**What this does:** Removes temporary files and organises results.

**⚠️ Warning:** This will delete intermediate files. Only run if you're sure you don't need them!

In [ ]:
# Set this to True only if you want to clean up temporary files
CLEAN_UP_FILES = False  # Change to True to enable cleanup

if CLEAN_UP_FILES:
    print("🧹 Starting cleanup process...")
    
    # List of directories that could be cleaned up
    cleanup_dirs = [
        # home_directory + 'embedding_data/raw_images/',  # Uncomment to delete raw images
        # home_directory + 'embedding_data/watermarked_images/',  # Uncomment to delete watermarked images
        # home_directory + 'embedding_data/transformed_images/',  # Uncomment to delete transformed images
    ]
    
    for cleanup_dir in cleanup_dirs:
        if os.path.exists(cleanup_dir):
            import shutil
            shutil.rmtree(cleanup_dir)
            print(f"🗑️ Deleted: {cleanup_dir}")
    
    print("✅ Cleanup complete!")
    
else:
    print("⏭️ Skipping cleanup (CLEAN_UP_FILES = False)")
    print("💡 All files have been preserved for your review")

# Final summary
print("\n" + "="*60)
print("🎉 WATERMARK TESTING PIPELINE COMPLETE!")
print("="*60)
print(f"📁 Results location: {results_dir}")
print(f"📊 Detection rate: {overall_detection_rate:.1f}%")
print(f"🔍 Images tested: {len(df_results)}")
print(f"🔄 Transformations: {len(transformations)}")
print("\n💡 Next steps:")
print("   • Review the CSV files for detailed results")
print("   • Check the charts for visual analysis")
print("   • Consider adjusting watermark parameters if needed")
print("   • Test with different image types or transformations")
print("\nThank you for using the Watermark Testing Pipeline! 🚀")